# 1. Agafar el text d'una web

In [48]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline
# %pip install hf_xet
# import torch
# %pip install torch


# # Descargar recursos necesarios de NLTK
nltk.download('punkt_tab')
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
def generate_summary(article_text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    
    # BART tiene límite de tokens, cortar si el texto es muy largo
    article_text = article_text[:1024]
    
    summary = summarizer(article_text, max_length=130, min_length=50, do_sample=False)
    return summary[0]['summary_text']

In [50]:
def huggingface_classifier(data):
    from transformers import pipeline

    # Crear pipeline
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Important si el text es en anglés, posar els topics també en anglés

    # Definició dels temes que pot tenir l'article
    # possible_labels = ["política", "deportes", "economía", "ciencia", "noticias","tecnología", "salud", "cultura", "guerra", "internacional"]
    
    possible_labels = ["politics", "sports", "economy", "science", "news", "technology", "health", "culture", "war", "international"]


    article_text = " ".join(data)[:500]
    result = classifier(article_text, candidate_labels=possible_labels)

    print("Tema más probable:", result['labels'][0])
    print("Scores:", list(zip(result['labels'], result['scores'])))

    # Retornar resultado completo por si se quiere usar después
    return result

In [ ]:
def title_article_preprocessing_a(clean_text):

    # 1. Toeknize del texte
    # tokens = word_tokenize(clean_text, language='spanish')
    tokens = word_tokenize(clean_text, language='english')
    
    # 2. Normalitzar a minúscules i eliminar puntuació
    tokens = [t.lower() for t in tokens if t.isalpha()]  # solo palabras
    
    # 3. Eliminar stopweords
    stop_words = set(stopwords.words('spanish'))
    tokens = [t for t in tokens if t not in stop_words]
    
    
    print(tokens[:50]) 

    return tokens


def title_article_preprocessing_b(clean_text):
    # ToDo: Implementar otra funcion de procesameinto de texto diferente 
    pass

In [52]:
import requests
from bs4 import BeautifulSoup

url = "https://link.springer.com/article/10.1186/s12915-020-00925-x"

opcio = input("Selecciona opció A o B")


def scrape_text(url, opcio):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 1. Intentar <article>
    article = soup.find("article")
    if article:
        text = article.get_text(separator="\n")
    else:
        # 2. Intentar <main>
        main = soup.find("main")
        if main:
            text = main.get_text(separator="\n")
        else:
            # 3. Fallback: todos los <p>, unir el bloque más largo
            paragraphs = soup.find_all("p")
            blocks = []
            current_block = []
            for p in paragraphs:
                current_block.append(p.get_text())
                # Separar bloques grandes por lógica simple
                if len(current_block) > 3:  
                    blocks.append("\n".join(current_block))
                    current_block = []
            if current_block:
                blocks.append("\n".join(current_block))
            # Elegir el bloque más largo (probable artículo)
            text = max(blocks, key=len) if blocks else ""
    
    # Limpiar líneas vacías
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    clean_text = "\n".join(lines)
    

    print(clean_text[:1000])  # Mostrar solo los primeros 1000 caracteres
    if opcio == "A":
        data = title_article_preprocessing_a(clean_text)
    elif opcio == "B":
        data = title_article_preprocessing_b(clean_text)
    else:
        print("Opció no vàlida")

    return data



    


In [53]:
def main():

    data = scrape_text(url, opcio)
    results = huggingface_classifier(data)
    print(results)
    text = generate_summary(" ".join(data))
    print("Summary:", text)

main()

Home
BMC Biology
Article
Comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite DNA repeats
Research article
Open access
Published:
21 December 2020
Volume 18
, article number
199
, (
2020
)
Cite this article
You have full access to this
open access
article
Download PDF
Save article
View saved research
BMC Biology
Aims and scope
Submit manuscript
Comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite DNA repeats
Download PDF
Octavio M. Palacios-Gimenez
ORCID:
orcid.org/0000-0002-1472-9949
1
,
2
,
Julia Koelman
1
,
Marc Palmada-Flores
1
,
Tessa M. Bradford
3
,
4
,
Karl K. Jones
3
,
Steven J. B. Cooper
3
,
4
,
Takeshi Kawakami
1
,
5
na1
&
…
Alexander Suh
1
,
2
,
6
na1
Show authors
6441
Accesses
47
Citations
12
Altmetric
Explore all metrics
Abstract
Background
Repetitive DNA sequences, including transposable elements (TEs) and tand

Device set to use cpu


Tema más probable: science
Scores: [('science', 0.6480868458747864), ('news', 0.09028579294681549), ('economy', 0.07996293902397156), ('technology', 0.04925379902124405), ('culture', 0.03950726240873337), ('health', 0.03777254372835159), ('international', 0.029102083295583725), ('war', 0.014171305112540722), ('sports', 0.007388153579086065), ('politics', 0.0044692582450807095)]
{'sequence': 'home bmc biology article comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite dna repeats research article open access published december volume article number cite this article you have full access to this open access article download pdf save article view saved research bmc biology aims and scope submit manuscript comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements a', 'labels': ['science', 'news', 'economy', 'technology', 'culture', 'health', 'international', 'war', 's

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   4%|3         | 62.9M/1.63G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Summary:  comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite dna repeats. Grasshoppers have large genomes averaging gb that contain high proporti. repetitive dna sequences are found in high proportion in organisms across the tree of life.
